# JSON Conversions

This notebook demonstrates how to convert a Resource to JSON and vice-versa. It can be used with one of the two stores: 
* The Demo Store, which is an in-memory store. Some features demonstrated in this notebook are not supported for this store. Do not use it for production.
* Nexus, which uses [Blue Bran Nexus](https://bluebrainnexus.io/) as a backend.

In [1]:
from kgforge.core import KnowledgeGraphForge

## Create the Forge using Demo Store and Demo Model

In [2]:
forge = KnowledgeGraphForge("../../configurations/demo-forge.yml")

## Create the Forge using Nexus Store and RDF Model

To run the demo using Nexus, use the next three cells. Provide the Nexus token and the organization and project you want to use.

In [ ]:
import getpass

In [ ]:
token = getpass.getpass()

In [ ]:
bucket = "dke/kgforge_tests"

In [ ]:
forge = KnowledgeGraphForge("../../configurations/demo-forge-nexus.yml", bucket=bucket, token=token)

# Imports

In [3]:
import json

In [4]:
from kgforge.core import Resource

In [5]:
def pp(x): print(json.dumps(x, indent=4))

## Resource to JSON

In [6]:
address = Resource(type="PostalAddress", country="Switzerland", locality="Geneva")

In [7]:
jane = Resource(type="Person", name="Jane Doe", address=address, email="(missing)")

In [8]:
john = Resource(type="Person", name="John Smith", email="john.smith@epfl.ch")

In [9]:
persons = [jane, john]

In [10]:
forge.register(persons)

<count> 2
<action> _register_one
<succeeded> True


In [11]:
association = Resource(type="Association", agent=forge.reshape(persons, keep=["id"]))

In [12]:
forge.register(association)

<action> _register_one
<succeeded> True


In [13]:
pp(forge.as_json(association))

{
    "id": "6e07113b-91dc-4b50-85f6-7d0930e3d235",
    "type": "Association",
    "agent": [
        {
            "id": "c4fddcd1-b6c7-48b8-8dab-61cedb7228bc"
        },
        {
            "id": "8ad1f2e7-4aae-4c37-8b65-241f2946e31b"
        }
    ]
}


In [14]:
pp(forge.as_json(jane, expanded=True))

{
    "@id": "file:///Users/agarcia/Developments/kgforge/examples/notebooks/getting-started/c4fddcd1-b6c7-48b8-8dab-61cedb7228bc",
    "@type": "schema:Person",
    "schema:name": "Jane Doe"
}


In [15]:
pp(forge.as_json(jane, store_metadata=True))

{
    "id": "c4fddcd1-b6c7-48b8-8dab-61cedb7228bc",
    "type": "Person",
    "address": {
        "type": "PostalAddress",
        "country": "Switzerland",
        "locality": "Geneva"
    },
    "email": "(missing)",
    "name": "Jane Doe",
    "deprecated": false,
    "version": 1
}


## JSON to Resource

In [16]:
address = Resource(type="PostalAddress", country="Switzerland", locality="Geneva")

In [17]:
jane = Resource(type="Person", name="Jane Doe", address=address, email="(missing)")

In [18]:
john = Resource(type="Person", name="John Smith", email="john.smith@epfl.ch")

In [19]:
persons = [jane, john]

In [20]:
association = Resource(type="Association", agent=[jane, john])

In [21]:
data = {
    "type": "Association",
    "agent": [
        {
            "type": "Person",
            "address": {
                "type": "PostalAddress",
                "country": "Switzerland",
                "locality": "Geneva",
            },
            "email": "(missing)",
            "name": "Jane Doe"
        },
        {
            "type": "Person",
            "email": "john.smith@epfl.ch",
            "name": "John Smith"
        }
    ]
}

In [22]:
resource = forge.from_json(data)

In [23]:
resource == association

True

In [24]:
print(forge.from_json(data, na="(missing)"))

{
    type: Association
    agent:
    [
        {
            type: Person
            address:
            {
                type: PostalAddress
                country: Switzerland
                locality: Geneva
            }
            name: Jane Doe
        }
        {
            type: Person
            email: john.smith@epfl.ch
            name: John Smith
        }
    ]
}
